# Run the simulation in Flex to generate the RAW data
Seems like the material is given by the particle's index. e.g. the first 64 particles are solid and the rest are the fluid.
Looks like we can tell by the phase.


- Save the fixed walls
For each time step:
- save the positions of particles and their materials
# Parameters
Parameters for BoxBath:
- Trajectory length: 150
- Number of rollouts: 2700 train, 150 Validation, 150 Test

In [ ]:
import h5py
import numpy as np
import pyflex
""" Planes
    (Vec4&)g_params.planes[0] = Vec4(up.x, up.y, up.z, 0.0f);
    (Vec4&)g_params.planes[1] = Vec4(0.0f, 0.0f, 1.0f, -g_sceneLower.z);
    (Vec4&)g_params.planes[2] = Vec4(1.0f, 0.0f, 0.0f, -g_sceneLower.x);
    (Vec4&)g_params.planes[3] = Vec4(-1.0f, 0.0f, 0.0f, g_sceneUpper.x);
    (Vec4&)g_params.planes[4] = Vec4(0.0f, 0.0f, -1.0f, g_sceneUpper.z);
    (Vec4&)g_params.planes[5] = Vec4(0.0f, -1.0f, 0.0f, g_sceneUpper.y);
"""
time_steps = 150
num_rollouts = 3000
f = h5py.File('./box_bath.hdf5', 'w')

pyflex.init()

for rollout in range(num_rollouts):
    print(f'Rollout {rollout}')
    # Set scene
    scene_params = np.zeros(1)
    pyflex.set_scene(1, scene_params, 0)
    num_rigid_positions = pyflex.get_n_rigidPositions()
    upper_bound = pyflex.get_scene_upper()
    lower_bound = pyflex.get_scene_lower()
    print(num_rigid_positions)
    print(upper_bound)
    print(lower_bound)
    # TODO: Do we need to clip the initial few steps
    num_particles = pyflex.get_n_particles()
    phases= np.zeros((time_steps, num_particles), dtype=np.float32)
    positions = np.zeros((time_steps, num_particles, 3), dtype=np.float32)
    for step in range(time_steps):
        phases[step] = pyflex.get_phases()
        positions[step] = pyflex.get_positions().reshape(-1,4)[:,:3]
        pyflex.step()
    f.create_dataset(f'rollouts/{rollout}/positions', data=positions, compression='gzip')
    f.create_dataset(f'rollouts/{rollout}/phases', data=phases, compression='gzip')
pyflex.clean()
f.close()